In [1]:
import os
import time
import numpy as np
import scipy.io as sio
from numpy.fft import fft2, ifft2, fftshift, ifftshift
import matplotlib.pyplot as plt
import utils
import torch

# Define centered FFT routines
def fft2c(x):
    return fftshift(fft2(ifftshift(x), axes=(0,1)))

def fft2c_cuda(x):
    return torch.fft.fftshift(torch.fft.fft2(torch.fft.ifftshift(x), dim=(0,1)))

def ifft2c(x):
    return fftshift(ifft2(ifftshift(x), axes=(0,1)))

def ifft2c_cuda(x):
    return torch.fft.fftshift(torch.fft.ifft2(torch.fft.ifftshift(x), dim=(0,1)))

# --- PARAMETERS ---

# Laser
wl = 0.532  # Wavelength in micrometers

# GPD (Diffuser)
gpd_dir = os.path.join('microretarders_mfile.mat')  # File containing diffuser data
input_pol = 'R'    # Polarization: 'R' (or 'L')
gpd_flip = 1       # Flip diffuser horizontally if needed
gpd_pix = 30       # Diffuser pixel size in micrometers
gpd_size = (512, 512)  # Dimensions of diffuser grid

# Camera
cam_flip = 0                  # Camera flip flag
cam_pix = 3.1                 # Camera pixel size in micrometers
cam_crop_size = (1024, 1024)     # Cropped sensor size

# Lenses
f1 = 300000     # Focal length of lens 1 in micrometers
f2 = 200000     # Focal length of lens 2 in micrometers
mag = f2 / f1   # System magnification
gamma_min = 9   # Minimum gamma threshold

# Display
display_pix = 36  # Display pixel size in micrometers
M_disp = display_pix / cam_pix * f2 / f1  # Display magnification factor
display_size = (768, 1024)
# Ensure display size does not exceed camera crop size:
display_size = (min(display_size[0], cam_crop_size[0]), min(display_size[1], cam_crop_size[1]))
effRectSize = (display_size[0] * display_pix / M_disp, display_size[1] * display_pix / M_disp)  # Effective sample FOV in um

# Gamma Checks
diffFOV = (gpd_size[0] * gpd_pix, gpd_size[1] * gpd_pix)  # Diffuser FOV in micrometers
N_max = np.prod(np.array(effRectSize) * np.array(diffFOV) / (wl * f1))  # Max resolvable elements
M_gpd = np.prod(gpd_size)  # Total diffuser pixels
M_camera = np.prod(np.array(cam_crop_size) * cam_pix * np.array(diffFOV) / (wl * f2))  # Camera resolution elements

gamma_gpd = M_gpd / N_max
gamma_diff = M_camera / N_max

print(f" | max samSize: [{effRectSize[0]*1e-3:.2f}, {effRectSize[1]*1e-3:.2f}] mm")
print(f" | gamma_gpd  > gamma_min: {gamma_gpd:.2f} > {gamma_min}")
print(f" | gamma_diff > gamma_min: {gamma_diff:.2f} > {gamma_min}")

 | max samSize: [3.57, 4.76] mm
 | gamma_gpd  > gamma_min: 1.66 > 9
 | gamma_diff > gamma_min: 1.33 > 9


In [2]:
# --- Generate Diffuser Field ---
anglescan = -0.19867  # Rotation angle for the diffuser (deg)
magscan = 0.9985      # Magnification correction factor
"""
diffuser_E, _, GPDwindow = utils.gpdField(
    gpd_dir, gpd_size, gpd_pix, gpd_flip, input_pol,
    cam_crop_size, cam_pix, cam_flip, wl, f1, f2, effRectSize,
    gpdRot=-anglescan, gpdMag=1/magscan
)
"""
diffuser_E = sio.loadmat('diffuser_E.mat')
diffuser_E = diffuser_E['diffuserE']
cam_pix2 = (cam_pix, cam_pix)

In [3]:
# --- Sample Parameters & Optimization Settings ---
is_sample_circle = 1    # 1: Circular aperture, 0: Square aperture
sam_FOV = 1.2e3     # Sample FOV in micrometers

optimization_method = 'FISTA'
correlation_critical = 1e-4

In [4]:
# Define Sample Mask (samMask)
padSize = diffuser_E.shape
if is_sample_circle:
    # Calculate radii in pixel units:
    rr = (round(sam_FOV * mag / cam_pix2[0] / 2), round(sam_FOV * mag / cam_pix2[1] / 2))
    # Create circular mask (note: mk_ellipse returns True outside the ellipse)
    sample_mask = ~utils.mk_ellipse(rr[1], rr[0], padSize[1], padSize[0])
else:
    sample_mask_size = (round(sam_FOV * mag / cam_pix2[0]), round(sam_FOV * mag / cam_pix2[1]))
    sample_mask = np.ones(sample_mask_size, dtype=bool)
    sample_mask = utils.mpad(sample_mask, padSize)
    
# Crop the sample mask to the display size for SLM mapping.
sample_mask_SLM = utils.mcrop(sample_mask, display_size)
# Compute row/column indices with nonzero mask sums.
mask1 = np.where(np.sum(sample_mask_SLM, axis=1) > 0)[0]
mask2 = np.where(np.sum(sample_mask_SLM, axis=0) > 0)[0]

# (In this translation we stick with numpy arrays. For GPU acceleration, consider using cupy.)
sample_mask_SLM = sample_mask_SLM.copy()
sample_mask = sample_mask.copy()

# temporary
pad_size = (1024, 1024)
spk_window = utils.mpad(np.ones(cam_crop_size, dtype=bool), pad_size)

# Bring diffuser field to "GPU" memory and apply ifftshift.
X_iter_0 = 'random'  # Initial guess type for reconstruction

# --- Ramp Calculation for SLM Pattern ---
Y_grid, X_grid = utils.ndgrid_matSizeIn(display_size, 0, 'centerZero')
ramp = 2 * np.pi * ((X_grid * 0.5) + (Y_grid * 0.5))
ramp = ramp.astype(np.float32)


In [5]:
# --- SLM Calibration ---
phaseVal, SLMDigit = utils.SLM_LUT()

def phase2digit(x):
    # Convert phase values to SLM digit values using nearest interpolation.
    return np.interp(x, phaseVal, SLMDigit)

shiftvec = (10, 5)

In [23]:
# from main_loop import main_loop
from main_loop import main_loop
from tqdm import tqdm
import numpy as np
import h5py

def load_mat_file(file_path):
    data = {}
    with h5py.File(file_path, 'r') as f:
        for key in f.keys():
            # Convert HDF5 datasets to numpy arrays.
            data[key] = np.array(f[key])
    return data

path = "rawImages-004.mat"
data = load_mat_file(path)
imgbuffer = data.get('imgbuffer')
imgbuffer = imgbuffer.transpose(1,2,0)

# Generate pupil-plane field image.
fields_pupil = np.zeros_like(imgbuffer, dtype=np.uint8)
fields = np.zeros_like(imgbuffer, dtype=np.uint8)

batch_size = 20

for i in tqdm(range(350, 370, batch_size), desc='processing...'):
    psi = imgbuffer[:,:,i:i+batch_size].astype(np.float32)
    psi = np.transpose(psi, axes=(1, 0, 2))
    psi = np.real(np.sqrt(psi))
    
    X_final = main_loop(psi, diffuser_E, sample_mask, spk_window, shiftvec)
    
    gpd_window = sio.loadmat('GPD_window.mat')
    gpd_window = gpd_window['GPDwindow']
    gpd_window = np.stack([gpd_window] * X_final.shape[2], axis=2)
    
    X_final = fft2c(np.roll(X_final, shift=(shiftvec[0], shiftvec[1]), axis=(0, 1))) * gpd_window
    # X_final = fft2c(X_final) * gpd_window
    X_final = ifft2c(X_final)

    absX = np.abs(X_final)
    perc = np.percentile(absX, 99.9)
    fields_pupil[:, :, i:i+batch_size] = (256 * absX / perc).astype(np.uint8)

    # Process again for the final output field image.
    X_final = ifft2c(X_final)
    absX = np.abs(X_final)
    perc = np.percentile(absX, 99.9)
    fields[:, :, i:i+batch_size] = (256 * absX / perc).astype(np.uint8)

processing...: 100%|██████████| 1/1 [00:13<00:00, 13.68s/it]


In [14]:
import time
from main_loop_speed import main_loop
from tqdm import tqdm
import numpy as np
import h5py
import scipy.io as sio

def load_mat_file(file_path):
    data = {}
    with h5py.File(file_path, 'r') as f:
        for key in f.keys():
            # Convert HDF5 datasets to numpy arrays.
            data[key] = np.array(f[key])
    return data

path = "rawImages-004.mat"
data = load_mat_file(path)
imgbuffer = data.get('imgbuffer')
imgbuffer = imgbuffer.transpose(1, 2, 0)

# Generate pupil-plane field image.
fields_pupil = np.zeros_like(imgbuffer, dtype=np.uint8)
fields = np.zeros_like(imgbuffer, dtype=np.uint8)

batch_size = 20

total_main_loop_time = 0
total_other_time = 0

for i in tqdm(range(0, imgbuffer.shape[2], batch_size), desc='processing...'):
    # start_other = time.time()
    psi = imgbuffer[:, :, i:i+batch_size].astype(np.float32)
    psi = np.transpose(psi, axes=(1, 0, 2))
    psi = np.real(np.sqrt(psi))

    # start_main = time.time()
    X_final = main_loop(psi, diffuser_E, sample_mask, spk_window, shiftvec)
    # end_main = time.time()

    # total_main_loop_time += (end_main - start_main)

    gpd_window = sio.loadmat('GPD_window.mat')
    gpd_window = gpd_window['GPDwindow']
    gpd_window = np.stack([gpd_window] * X_final.shape[2], axis=2)
    
    gpd_window_torch = torch.tensor(gpd_window).cuda()

    X_final = fft2c_cuda(torch.roll(X_final, shifts=(shiftvec[0], shiftvec[1]), dims=(0, 1))) * gpd_window_torch
    X_final = ifft2c_cuda(X_final)
    
    # Process again for the final output field image.
    X_final = ifft2c_cuda(X_final)
    X_final = X_final.cpu().numpy()
    
    absX = np.abs(X_final)
    perc = np.percentile(absX, 99.9)
    fields[:, :, i:i+batch_size] = (256 * absX / perc).astype(np.uint8)
    
    
    absX = np.abs(X_final)
    perc = np.percentile(absX, 99.9)
    fields_pupil[:, :, i:i+batch_size] = (256 * absX / perc).astype(np.uint8)

    # end_other = time.time()
    # total_other_time += (end_other - start_other) - (end_main - start_main)

# print(f"Total time spent in main_loop: {total_main_loop_time:.2f} seconds")
# print(f"Total time spent in other operations: {total_other_time:.2f} seconds")


processing...: 100%|██████████| 105/105 [16:08<00:00,  9.23s/it]


In [15]:
import h5py

# Save to HDF5
with h5py.File('fields.h5', 'w') as f:
    f.create_dataset('fields', data=fields)
    f.create_dataset('fields_pupil', data=fields_pupil)

In [16]:
f.close()

In [17]:
import napari

viewer = napari.Viewer()
viewer.add_image(fields)

<Image layer 'fields' at 0x2dff9c16ae0>

c:\miniconda3\envs\tele\Lib\site-packages\napari\_vispy\layers\scalar_field.py:198: UserWarning: data shape (1024, 1024, 2100) exceeds GL_MAX_TEXTURE_SIZE 2048 in at least one axis and will be downsampled. Rendering is currently in 3D mode.
  warnings.warn(
